In [ ]:
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True)
%cd /content/gdrive/My Drive/Project FPAR2/ego-rnn


  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=68adcba606cab51717e592fe6f1ef0e37ad030ce653ed21940bd734f077589ae
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/gdrive
/content/gdrive/My Drive/Project FPAR2/ego-rnn


In [2]:

from __future__ import print_function, division
from objectAttentionModelConvAugmentedLSTM import *
#from objectAttentionModelConvLSTMVisualize import *
from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
!pip install tensorboardX
from tensorboardX import SummaryWriter
#from MyConvGruCell import *
from makeDatasetRGBSelection import *

import argparse
import sys
from resnetMod import *
from plotting import *

def main_run(dataset, stage, train_data_dir, val_data_dir, stage1_dict, out_dir, seqLen, trainBatchSize,
             valBatchSize, numEpochs, lr1, decay_factor, decay_step, memSize, cam):

    if dataset == 'gtea61':
        num_classes = 61
    elif dataset == 'gtea71':
      num_classes = 71
    elif dataset == 'gtea_gaze':
        num_classes = 44
    elif dataset == 'egtea':
        num_classes = 106
    else:
        print('Dataset not found')
        sys.exit()

    model_folder = out_dir  # Dir for saving models and log files
    

    # Log files
    writer = SummaryWriter(model_folder)
    train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
    train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
    val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
    val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')


    # Data loader
    normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224), ToTensor(), normalize])
    from torchvision.transforms import Resize
    spatial_transform_mmap = Compose([Resize(256),ToTensor()])
    vid_seq_train = makeDataset(train_data_dir,
                                spatial_transform=spatial_transform, seqLen=seqLen, fmt='.png')

    train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                            shuffle=True, num_workers=4, pin_memory=True)
    if val_data_dir is not None:

        vid_seq_val = makeDataset(val_data_dir,
                                   spatial_transform=Compose([Scale(256), CenterCrop(224), ToTensor(), normalize]),
                                   seqLen=seqLen, train=False, fmt='.png')

        val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize,
                                shuffle=False, num_workers=2, pin_memory=True)
        valInstances = vid_seq_val.__len__()


    trainInstances = vid_seq_train.__len__()

    train_params = []
    if stage == 1:

        model = attentionModel(num_classes=num_classes, mem_size=memSize)
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False
    else:

        model = attentionModel(num_classes=num_classes, mem_size=memSize)
        model.load_state_dict(torch.load(stage1_dict))
        model.train(False)
        for params in model.parameters():
            params.requires_grad = False
        #
        for params in model.resNet.layer4[0].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[0].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv1.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[1].conv2.parameters():
            params.requires_grad = True
            train_params += [params]

        for params in model.resNet.layer4[2].conv1.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.layer4[2].conv2.parameters():
            params.requires_grad = True
            train_params += [params]
        #
        for params in model.resNet.fc.parameters():
            params.requires_grad = True
            train_params += [params]

        model.resNet.layer4[0].conv1.train(True)
        model.resNet.layer4[0].conv2.train(True)
        model.resNet.layer4[1].conv1.train(True)
        model.resNet.layer4[1].conv2.train(True)
        model.resNet.layer4[2].conv1.train(True)
        model.resNet.layer4[2].conv2.train(True)
        model.resNet.fc.train(True)

    for params in model.lstm_cell.parameters():
        params.requires_grad = True
        train_params += [params]

    for params in model.classifier.parameters():
        params.requires_grad = True
        train_params += [params]

    model.lstm_cell.train(True)
    model.classifier.train(True)
    model.cuda()

    loss_fn = nn.CrossEntropyLoss()

    optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)

    optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step,
                                                           gamma=decay_factor)
    
    train_loss_list=[]
    val_loss_list=[]
    train_acc_list=[]
    val_acc_list=[]

    train_iter = 0
    min_accuracy = 0
    print('train')

    for epoch in range(numEpochs):
        optim_scheduler.step()
        epoch_loss = 0
        numCorrTrain = 0
        trainSamples = 0
        iterPerEpoch = 0
        model.lstm_cell.train(True)
        model.classifier.train(True)
        writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)
        if stage == 2:
            model.resNet.layer4[0].conv1.train(True)
            model.resNet.layer4[0].conv2.train(True)
            model.resNet.layer4[1].conv1.train(True)
            model.resNet.layer4[1].conv2.train(True)
            model.resNet.layer4[2].conv1.train(True)
            model.resNet.layer4[2].conv2.train(True)
            model.resNet.fc.train(True)
            
        for i, (inputs, targets) in enumerate(train_loader):
            train_iter += 1
            iterPerEpoch += 1
            optimizer_fn.zero_grad()
            inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda())
            labelVariable = Variable(targets.cuda())
            trainSamples += inputs.size(0)
            output_label, _ = model(inputVariable)
            #output_label, _ = model(inputVariable)
            loss = loss_fn(output_label, labelVariable)
            loss.backward()
            optimizer_fn.step()
            _, predicted = torch.max(output_label.data, 1)
            numCorrTrain += (predicted == targets.cuda()).sum().item()
            epoch_loss += loss.data
        avg_loss = epoch_loss/iterPerEpoch
        trainAccuracy = (numCorrTrain / trainSamples) * 100

        train_loss_list.append(avg_loss)
        train_acc_list.append(trainAccuracy)

        print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch+1, avg_loss, trainAccuracy))
        writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
        writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)
        if val_data_dir is not None:
            if (epoch+1) % 1 == 0:
                model.train(False)
                val_loss_epoch = 0
                val_iter = 0
                val_samples = 0
                numCorr = 0
                for j, (inputs, targets) in enumerate(val_loader):
                    val_iter += 1
                    val_samples += inputs.size(0)
                    inputVariable = Variable(inputs.permute(1, 0, 2, 3, 4).cuda(), volatile=True)
                    labelVariable = Variable(targets.cuda(async=True), volatile=True)
                    output_label, _ = model(inputVariable)
                    val_loss = loss_fn(output_label, labelVariable)
                    val_loss_epoch += val_loss.data
                    _, predicted = torch.max(output_label.data, 1)
                    numCorr += (predicted == targets.cuda()).sum().item()
                val_accuracy = (numCorr / val_samples) * 100
                avg_val_loss = val_loss_epoch / val_iter

                val_loss_list.append(avg_val_loss)
                val_acc_list.append(val_accuracy)
                print('Val: Epoch = {} | Loss {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
                writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
                writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
              
                if val_accuracy > min_accuracy:
                    save_path_model = (model_folder + '/model_rgb_state_dict.pth')
                    torch.save(model.state_dict(), save_path_model)
                    min_accuracy = val_accuracy
            else:
                if (epoch+1) % 10 == 0:
                  i=0
                  save_path_model = (model_folder + '/model_rgb_state_dict_epoch' + str(epoch+1) + '.pth')
                  torch.save(model.state_dict(), save_path_model)
    '''
    train_log_loss.close()
    train_log_acc.close()
    val_log_acc.close()
    val_log_loss.close()
    writer.export_scalars_to_json(model_folder + "/all_scalars.json")
    writer.close()
    '''
    plotting(train_loss_list, val_loss_list,train_acc_list, val_acc_list)


def __main__():

    main_run(dataset='gtea61', stage=1, train_data_dir='./GTEA61/processed_frames2', val_data_dir='./GTEA61/processed_frames2', stage1_dict='./experiments/selection+augm/stage1/model_rgb_state_dict.pth', out_dir='experiments/selection+augm/stage2', seqLen=16, trainBatchSize=32,
             valBatchSize=64, numEpochs=150, lr1=1e-4, decay_factor=0.1, decay_step=[25,75,150], memSize=512, cam=True)

__main__()

importing Jupyter notebook from objectAttentionModelConvAugmentedLSTM.ipynb
importing Jupyter notebook from resnetMod.ipynb
importing Jupyter notebook from MyConvAugmentedLSTMCell.ipynb
importing Jupyter notebook from spatial_transforms.ipynb
     |████████████████████████████████| 317kB 5.7MB/s 
importing Jupyter notebook from makeDatasetRGBSelection.ipynb
importing Jupyter notebook from plotting.ipynb


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


MyConvAugmentedLSTMCell.ipynb:50: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:51: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:52: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:54: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:55: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:56: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:58: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:59: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:60: UserWarning:

train


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Train: Epoch = 1 | Loss = 4.108048915863037 | Accuracy = 4.424778761061947


KeyboardInterrupt: ignored

In [ ]:
!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True)
%cd /content/gdrive/My Drive/Project FPAR2/ego-rnn

num_classes=61
memSize=512

from countParameters import *
from objectAttentionModelConvAugmentedLSTM import *
model = attentionModel(num_classes=num_classes, mem_size=memSize)
from objectAttentionModelLSTA import *

model2 = attentionModel(num_classes=num_classes, mem_size=memSize)

n1=count_parameters(model)
n2=count_parameters(model2)
print(n1)
print(n2)

Mounted at /content/gdrive
/content/gdrive/My Drive/Project FPAR2/ego-rnn
importing Jupyter notebook from countParameters.ipynb
importing Jupyter notebook from objectAttentionModelConvAugmentedLSTM.ipynb
importing Jupyter notebook from resnetMod.ipynb
importing Jupyter notebook from MyConvAugmentedLSTMCell.ipynb


MyConvAugmentedLSTMCell.ipynb:50: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:51: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:52: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:54: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:55: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:56: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:58: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvAugmentedLSTMCell.ipynb:59: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvAugmentedLSTMCell.ipynb:60: UserWarning:

importing Jupyter notebook from objectAttentionModelLSTA.ipynb
importing Jupyter notebook from MyConvLSTACell.ipynb
importing Jupyter notebook from resNetNew.ipynb


MyConvLSTACell.ipynb:49: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:50: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:51: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:53: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:54: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:55: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:57: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:58: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:59: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyCo

+-------------------------------------+------------+
|               Modules               | Parameters |
+-------------------------------------+------------+
|            weight_softmax           |   512000   |
|         resNet.conv1.weight         |    9408    |
|          resNet.bn1.weight          |     64     |
|           resNet.bn1.bias           |     64     |
|     resNet.layer1.0.conv1.weight    |   36864    |
|      resNet.layer1.0.bn1.weight     |     64     |
|       resNet.layer1.0.bn1.bias      |     64     |
|     resNet.layer1.0.conv2.weight    |   36864    |
|      resNet.layer1.0.bn2.weight     |     64     |
|       resNet.layer1.0.bn2.bias      |     64     |
|     resNet.layer1.1.conv1.weight    |   36864    |
|      resNet.layer1.1.bn1.weight     |     64     |
|       resNet.layer1.1.bn1.bias      |     64     |
|     resNet.layer1.1.conv2.weight    |   36864    |
|      resNet.layer1.1.bn2.weight     |     64     |
|       resNet.layer1.1.bn2.bias      |     64

MyConvLSTACell.ipynb:105: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:106: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:108: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:109: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:112: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTACell.ipynb:113: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTACell.ipynb:114: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.


In [ ]:
for i in np.linspace(1, 60, 7, endpoint=False):
  print(np.floor(i))

1.0
9.0
17.0
26.0
34.0
43.0
51.0
